In [ ]:
import numpy as np

import os

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import model_from_json
from scipy.stats import pearsonr

from mask_utils import show_image_with_masks,iou,symmetric_hausdorff_distance,mean_contour_distance,dsc

from network_utils import gpu_memory_limit

from MultiResUNet.MultiResUNet import MultiResUnet

import itertools

In [ ]:
#limit how much GPU RAM can be allocated by this notebook... 8GB is 1/3 of available
gpu_memory_limit(6000)

In [ ]:
DataDir = './data/pericardial/wsx_round2/'

#load data - these files created by extract_dcm_for_wsx.ipynb
X = np.load(os.path.join(DataDir,'X.npy'))
Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')
pxArea = np.load(os.path.join(DataDir,'pxSize.npy'))
pxSpacing = np.sqrt(pxArea)

#ensure the shape is correct arrays saved were rank 3, so this changes to rank 4 (last dimension represents channels)
X = X.reshape([*X.shape,1])
Y = Y.reshape([*Y.shape,1])

#do train/test split!
X, X_test, Y, Y_test,pxArea,pxArea_test,pxSpacing,pxSpacing_test = train_test_split(X, Y, pxArea,pxSpacing, test_size=0.2,random_state=101)

#
M = X.shape[0]
MTest = X_test.shape[0]

In [ ]:
#load the model
modelBaseName = 'mrunet_bayesian_2020-06-15_17:46' 

modelBaseName = os.path.join('data','models',modelBaseName)

modelParamFile = modelBaseName + '.h5'
modelArchitecture = modelBaseName + '.json'

with open( modelArchitecture , 'r') as json_file:
    model = model_from_json( json_file.read() )

model.load_weights(modelParamFile)

model_noDropout = MultiResUnet(height=X.shape[1],
                               width=X.shape[2],
                               n_channels=1,
                               layer_dropout_rate=None,
                               block_dropout_rate=0,
                              ).load_weights(modelParamFile)

In [ ]:
#FUNCTIONS FOR DOING STOCHASTIC PREDICTIONS...

#FIXMMEEEEEEEE make it so these can be called on arrays where M>1!!!!! BECAUSE THIS SUCKS

def global_iou(predictions):
    
    '''takes the iou of multiple different segmentations'''
    
    intersection = np.min(predictions,axis=0).sum()
    union = np.max(predictions,axis=0).sum()
    
    return intersection / union

def global_dsc(predictions):
    
    N = predictions.shape[0]
    numerator = N * np.min(predictions,axis=0).sum()
    denominator = predictions.sum()
    
    return numerator/denominator
    
def mean_pairwise_iou(predictions):
    
    #all combinations of inputs
    ious = [iou(a,b) for a,b in itertools.combinations(predictions,2)]
    
    return np.mean(ious)

def mean_pairwise_dsc(predictions):
    
    #all combinations of samples, which will be axis 0
    dscs = [dsc(a,b) for a,b in itertools.combinations(predictions,2)]
    
    return np.mean(dscs)
    
def voxel_uncertainty(predictions):
    
    '''voxel-wise uncertainty as defined in Roy et al (2018)'''
    
    #strcture-and-voxel-wise uncertainty (compresses over the sample axis
    feature_uncertainty = -np.sum(predictions*np.log(predictions),axis = 0)
    #global uncertainty is the sum over the feature axis
    global_uncertainty = np.sum(feature_uncertainty,axis=-1)
    
    return global_uncertainty
    
def mean_std_area(predictions):
    
    '''the area occupied by each segmented channel. outputs two array: mean and standard deviation
    RETURNS ANSWERS IN PIXELS WHICH MUST BE RESCALED LATER!!!!!!
    '''
    #get the dims
    N = predictions.shape[0]
    nPixels = np.product(predictions.shape[1:-1])
    nFeatures = predictions.shape[-1]
    
    #reshape array so that it is (N,pixels,features) and thrshold.
    predictions = predictions.reshape((N,nPixels,nFeatures)) > 0.5
    
    #sum of voxels for each 
    areas = np.sum(predictions,axis = 1)
    
    #mean, returning a value for each segmentation channel
    mu = np.mean(areas,axis=0)
    sigma = np.std(areas,axis=0)
    
    return mu,sigma

def predict_stochastic(model,N,X):
    
    '''draw and summarise multiple predictions from a model
    Arguments:
        model {a model, for example a Keras model, with a predict method} -- is assumed to have some stochastic component, i.e. multiple
        N {int} -- the number of sample predictions to be drawn from the stochastic model
        X {numpy array, probably float} -- assumed to be already consistent with inputs to the model. MUST ONLY BE A SINGLE IMAGE AND NOT MULTIPLE STACKED!!!!!
        
    Returns:
        consensus {numpy array, boolean} -- pixelwise segmentation of x
        also various floats, representing different metrics for uncertainty and the outputs.
    '''
    
    #draw N predictions from the model over x
    predictions = np.stack([model.predict(X) for n in range(N)],axis=0)
    
    #binarise
    predictions = predictions
    
    consensus = np.mean(predictions,axis=0)>0.5 
    
    #metrics described in Roy et al...
    uncertainty = voxel_uncertainty(predictions)
    
    mpDsc = mean_pairwise_dsc(predictions)
    gDsc = global_dsc(predictions)
    
    mpIou = mean_pairwise_iou(predictions)
    gIou = global_iou(predictions)
    meanArea,stdArea = mean_std_area(predictions)
    
    return consensus,uncertainty,meanArea,stdArea,mpDsc,gDsc,mpIou,gIou

First, lets go through a range of values for N, the number of samples drawn in making a prediction, before manually selecting an optimal value.

In [ ]:
NRange = np.arange(2,12,2)

iouMean = np.zeros_like(NRange)
dscMean = np.zeros_like(NRange)
iouStd = np.zeros_like(NRange)
dscStd = np.zeros_like(NRange)
areaStdMean = np.zeros_like(NRange)
areaStdStd = np.zeros_like(NRange)

for ind,N in enumerate(NRange):

    predTest,uncertaintyTest,meanAreaTest,stdAreaTest,mpDscTest,gDscTest,mpIouTest,gIouTest = map(np.array,zip(*[predict_stochastic(model,N,x.reshape(1,208,208,1)) for x in X_test]))
    
    predTest = predTest.reshape(*Y_test.shape)
    #loop over th eexample axis, calculating metrics for each image separately
    TestIOU = [iou(Y_test[m,:,:,:], predTest[m,:,:]) for m in range(MTest)]
    TestDSC = [dsc(Y_test[m,:,:,:], predTest[m,:,:]) for m in range(MTest)]
    
    iouMean[ind] = np.mean(TestIOU)
    dscMean[ind] = np.mean(TestDSC)
    
    iouStd[ind] = np.std(TestIOU)
    dscStd[ind] = np.std(TestDSC)
    
    areaStdMean[ind] = stdAreaTest.mean()
    areaStdStd[ind] =stdAreaTest.std()

#get the benchmark for model without dropout...
predDeterministic = model_noDropout.predict(X_test)
#loop over th eexample axis, calculating metrics for each image separately
TestIOU_Deterministic = [iou(Y_test[m,:,:,:], predDeterministic[m,:,:]) for m in range(MTest)]
TestDSC_Deterministic = [dsc(Y_test[m,:,:,:], predDeterministic[m,:,:]) for m in range(MTest)]

iouMean_Deterministic = np.mean(TestIOU_Deterministic)
dscMean_Deterministic = np.mean(TestDSC_Deterministic)
iouStd_Deterministic = np.std(TestIOU_Deterministic)
dscStd_Deterministic = np.std(TestDSC_Deterministic)


In [ ]:


#set the graph x location for 
detX = NRange.max() + 2

errorbarArgs = {'capsize':3,
                'marker':'o'
               }

xt = np.arange(NRange.min(),NRange.max()+1,5)
xTicks = xt.tolist().append(detX)
xNames = xt.tolist().append('No Dropout')
xTickLabels = dict(zip(xTicks,xNames))

plt.figure(figsize = (15,5))

plt.subplot(1,3,1)
plt.errorbar(NRange,iouMean,iouStd,**errorbarArgs)
plt.errorbar(detX,iouMean_Deterministic,iouStd_Deterministic,**errorbarArgs)
plt.xlabel('Monte Carlo N')

plt.subplot(1,3,2)
plt.errorbar(NRange,dscMean,dscStd)
plt.errorbar(detX,dscMean_Deterministic,dscStd_Deterministic)
plt.xlabel('Monte Carlo N')

plt.subplot(1,3,3)
plt.errorbar(NRange,areaStdMean,areaStdStd)
plt.xlabel('Monte Carlo N')
plt.ylabel()

plt.savefig(modelBaseName + '_sample_size.png')
plt.savefig(modelBaseName + '_sample_size.svg')


And so, the correct answer is to use a sample size which gives near-best performance _AND_ spread (less samples is obvs better tho).

In [ ]:
N = 20

predTest,uncertaintyTest,meanAreaTest,stdAreaTest,mpDscTest,gDscTest,mpIouTest,gIouTest = map(np.array,zip(*[predict_stochastic(model,N,x.reshape(1,208,208,1)) for x in X_test]))

predTest = predTest.reshape(*Y_test.shape)
